# Pipeline 2018-2019
Basado en `generacion_features`, esta notebook ejecuta un pipeline similar para generar los indicadores para una base nueva de 2018 y 2019.

In [8]:
import pandas as pd

# 1. Preprocesamiento

Antes de trabajar con las bases de datos, necesitamos llenar campos vacíos, transformar los tipos de datos, y reproyectar las tablas a un formato más eficiente.

## 1.1 Procedimientos

Utilizando `src/utils/clean_data.py` tenemos que trabajar las bases de datos de [contratos de Compranet](https://sites.google.com/site/cnetuc/descargas) para limpiarlas, mezclarlas, y guardarlas en formato parquet. Vas a necesitar también `Directorio de unidades compradoras` y `Códigos repetidos en la tabla de procedimientos` de [imco/IRC#datos-procesados](https://github.com/imco/IRC/#datos-procesados).

In [1]:
from utils.clean_data import ensamblar_procedimientos

In [2]:
base_path = '../data/bases'
years = [2018, 2019]
files = [
    'Contratos2018.xlsx',
    'Contratos2019_200525070452.xlsx'
]

In [3]:
%%time
procs = ensamblar_procedimientos(files, years, base_path)

(383903, 53)
(383903, 46)
CPU times: user 8.54 s, sys: 1.59 s, total: 10.1 s
Wall time: 2min 7s


In [60]:
procs.groupby('FECHA_ARCHIVO').size()

FECHA_ARCHIVO
2018    194191
2019    189654
dtype: int64

In [61]:
%%time
procs.to_csv(f'{base_path}/procedimientos-v2018.psv', sep='|', encoding='utf-8', index=False, quoting=1)
procs.to_parquet(f'{base_path}/procedimientos-v2018.parquet', engine='pyarrow')

CPU times: user 8.91 s, sys: 369 ms, total: 9.28 s
Wall time: 9.3 s


In [70]:
!ls -lh ../data/bases/*v2018* | cut -d' ' -f5-

 55M Jun  1 20:18 ../data/bases/procedimientos-v2018.parquet
309M Jun  1 20:18 ../data/bases/procedimientos-v2018.psv


# 2. Cargar bases

In [13]:
df_procs = pd.read_parquet(f'{base_path}/procedimientos-v2018.parquet', engine='pyarrow')
df_procs.head()

,GOBIERNO,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,...,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,IMPORTE_PESOS
0,APF,CENTRO NACIONAL DE CONTROL DEL GAS NATURAL,CENAGAS,CENAGAS-UNIDAD DE FINANZAS Y ADMINISTRACION DI...,018TON999,ADRIAN MERCADO ZEPEDA,NO MIPYME,367,EMERSON PROCESS MANAGEMENT,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,101629.0
1,APF,FONATUR MANTENIMIENTO TURISTICO SA DE CV,FONATUR MANTENIMIENTO,FONATUR MANTENIMIENTO-GERENCIA CENTRAL SUBDIRE...,021W3S002,FERNANDO PORFIRIO LORENZO VAZQUEZ,NO MIPYME,367,EMERSON PROCESS MANAGEMENT,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,730996.8
2,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-DELEGACION ESTATAL DEL ISSSTE EN BAJA C...,051GYN052,REYNALDO PERAZA PERAZA,PEQUENA,485074,LUIS ERNESTO MEZA FLORES,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,272000.0
3,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-DELEGACION ESTATAL DEL ISSSTE EN BAJA C...,051GYN052,REYNALDO PERAZA PERAZA,PEQUENA,485074,LUIS ERNESTO MEZA FLORES,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,269000.0
4,APF,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,ISSSTE,ISSSTE-JEFATURA DE SERVICIOS DE ADQUISICION DE...,051GYN007,ANTONIO RAMIREZ GUTIERREZ,NO MIPYME,None,LABORATORIOS DE BIOLOGICOS Y REACTIVOS DE MEXICO,N/A,...,None,N/A,https://compranet.hacienda.gob.mx/esop/guest/g...,-1,MX,,None,None,2018,64940400.0
